In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/02 17:44:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Q1:** What is the spark version?

In [4]:
spark.version

'3.3.2'

In [10]:
!ls -lh fhv_tripdata_2019-10.csv.gz

-rw-rw-r-- 1 jackm jackm 19M Dec  2  2022 fhv_tripdata_2019-10.csv.gz


In [11]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('affiliated_base_number', types.StringType(), True),    
])

**Q2:** average size of the parquet partitions?

In [14]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

df = df.repartition(6)

df.write.parquet('data/pq/fhvhv/2019/10/')

In [15]:
df = spark.read.parquet('data/pq/fhvhv/2019/10/')

**Q3**: How many taxi trips were there on February 15?

In [16]:
from pyspark.sql import functions as F

In [23]:
df.registerTempTable('fhvhv_2019_10')

/home/jackm/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [24]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



**Q4**: Longest trip for each day

In [34]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropOff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhvhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
+-----------+------------------+



**Q6**: Most common locations pair

In [37]:
df_zones = spark.read.parquet('data/zones')

In [40]:
df_zones.registerTempTable('zones')

In [46]:
spark.sql("""
SELECT
    zone,
    COUNT(1)
    
  FROM fhvhv_2019_10 AS fh 
  LEFT JOIN zones AS zo
  ON fh.PULocationID = zo.LocationID
  
GROUP BY 
    1
ORDER BY
    2 asc
LIMIT 5;
""").show()

+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

